Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-credit-card-fraud/auto-ml-classification-credit-card-fraud.png)

# Automated Machine Learning
_**Classification of credit card fraudulent transactions on remote compute **_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Train](#Train)
1. [Results](#Results)
1. [Test](#Test)
1. [Acknowledgements](#Acknowledgements)

## Introduction

In this example we use the associated credit card dataset to showcase how you can use AutoML for a simple classification problem. The goal is to predict if a credit card transaction is considered a fraudulent charge.

This notebook is using remote compute to train the model.

If you are using an Azure Machine Learning Compute Instance, you are all set. Otherwise, go through the [configuration](../configuration.ipynb) notebook first if you haven't already to establish your connection to the AzureML Workspace. 

In this notebook you will learn how to:
1. Create an experiment using an existing workspace.
2. Configure AutoML using `AutoMLConfig`.
3. Train the model using remote compute.
4. Explore the results.
5. Test the fitted model.

## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For Automated ML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [8]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [9]:
print("This notebook was created using version 1.29 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.29 of the Azure ML SDK
You are currently using version 1.34.0 of the Azure ML SDK


In [10]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'test-set-support-automl-classification-ccard-remote'

experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,381b38e9-9840-4719-a5a0-61d9585e1e91
Workspace,cesardl-automl-ncentralus-demo-ws
Resource Group,cesardl-automl-ncentralus-demo-ws-resgrp
Location,northcentralus
Experiment Name,test-set-support-automl-classification-ccard-remote


## Create or Attach existing AmlCompute
A compute target is required to execute the Automated ML run. In this tutorial, you create AmlCompute as your training compute resource.
#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Data

### Load Data

Load the credit card dataset from a csv file containing both training features and labels. The features are inputs to the model, while the training labels represent the expected output of the model. Next, we'll split the data using random_split and extract the training data for the model.

In [12]:
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/creditcard.csv"
dataset = Dataset.Tabular.from_delimited_files(data)
training_data, test_data = dataset.random_split(percentage=0.8, seed=223)
label_column_name = 'Class'

## Train

Instantiate a AutoMLConfig object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**enable_early_stopping**|Stop the run if the metric score is not showing improvement.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|

**_You can find more information about primary metrics_** [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)

In [19]:
automl_settings = {
    "primary_metric": 'average_precision_score_weighted',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4, # This is a limit for testing purpose, please increase it as per cluster size
    "iterations": 10,
    "experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ablity to find the best model possible
    "verbosity": logging.INFO,
}


automl_config = AutoMLConfig(task = 'classification',
                             compute_target = compute_target,
                             training_data = training_data,
                             label_column_name = label_column_name,
                             
                             # Use validation set split
                             validation_size=0.2,
                             
                             # Use test set split (This will be used only for the winer model Test metrics)
                             test_size=0.1,
                             
                             **automl_settings
                            )

Call the `submit` method on the experiment object and pass the run configuration. Depending on the data and the number of iterations this can run for a while. Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [20]:
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
test-set-support-automl-classification-ccard-remote,AutoML_3422f595-4af4-4b2f-872e-1b1bfbb1cb82,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+==========================

In [21]:
# If you need to retrieve a run that already started, use the following code
#from azureml.train.automl.run import AutoMLRun
#remote_run = AutoMLRun(experiment = experiment, run_id = '<replace with your run id>')

## Results

#### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details

In [22]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
remote_run.wait_for_completion(show_output=False)

{'runId': 'AutoML_3422f595-4af4-4b2f-872e-1b1bfbb1cb82',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-10-25T21:57:36.694565Z',
 'endTimeUtc': '2021-10-25T22:13:18.143823Z',
 'services': {},
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'average_precision_score_weighted',
  'train_split': '0.2',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"test-set-support-automl-classification-ccard-remote","subscription_id":"381b38e9-9840-4719-a5a0-61d9585e1e91","resource_group":"cesardl-automl-ncentralus-demo-ws-resgrp","workspace_name":"cesardl-automl-ncentralus-demo-ws","region":"northcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_

#### Explain model

Automated ML models can be explained and visualized using the SDK Explainability library. 

## Analyze results

### Retrieve the Best Model/Run

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [26]:
best_run, fitted_model = remote_run.get_output()
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/m...
), random_state=None))], verbose=False)), ('4', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.4955555555555555, learning_rate=0.09473736842105263, max_bin=20, max_depth=4, min_child_weight=0, min_data_in_leaf=0.05517689655172415, min_split_gain=0.7894736842105263, n_estimators=10, n_jobs=1, num_leaves=176, problem_info=ProblemInfo(
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0.368421

#### Print the properties of the model
The fitted_model is a python object and you can read the different properties of the object.


#### Get the Predictions and Metrics Generated by the Test Run

Get the test run associated with the best run. When a remote test run is requested by passing in a value for `test_data` or `test_size` to `AutoMLConfig`, only the best training run will have a test run associated with it. To start a test run for models which are not associated with the best run or to start another best run test run, use `ModelProxy`. See the example later in the notebook in the __Test the fitted model remotely__ section for more details.

To see more details about the test run in Azure Machine Learning Studio (view its metrics, get a preview of the predictions, etc...) follow the link to the __Details Page__ listed in the next cells output.

In [27]:
test_run = next(best_run.get_children(type='automl.model_test'))
test_run.wait_for_completion(show_output=False, wait_post_processing=True)
test_run

Experiment,Id,Type,Status,Details Page,Docs Page
test-set-support-automl-classification-ccard-remote,e2fd2f41-0882-4bd1-877e-6fd8f73ace46,automl.model_test,Completed,Link to Azure Machine Learning studio,Link to Documentation


Get the __metrics__ from the test run.

In [28]:
test_run_metrics = test_run.get_metrics()
for name, value in test_run_metrics.items():
    print(f"{name}: {value}")

matthews_correlation: 0.7505032095637124
average_precision_score_macro: 0.8878577879244975
norm_macro_recall: 0.7889458404522789
recall_score_macro: 0.8944729202261394
recall_score_weighted: 0.9991217284384332
AUC_micro: 0.9996670111975998
precision_score_macro: 0.856966878260323
f1_score_weighted: 0.9991436465915597
precision_score_weighted: 0.9991718535619092
weighted_accuracy: 0.9994715695224134
AUC_weighted: 0.9598653535395628
average_precision_score_micro: 0.9995664236282711
balanced_accuracy: 0.8944729202261394
recall_score_micro: 0.9991217284384332
f1_score_macro: 0.874780045750484
precision_score_micro: 0.9991217284384332
f1_score_micro: 0.9991217284384332
accuracy: 0.9991217284384332
average_precision_score_weighted: 0.9995316154124612
AUC_macro: 0.9598653535395628
log_loss: 0.08762644383474373
confusion_matrix: aml://artifactId/ExperimentRun/dcid.e2fd2f41-0882-4bd1-877e-6fd8f73ace46/confusion_matrix
accuracy_table: aml://artifactId/ExperimentRun/dcid.e2fd2f41-0882-4bd1-877e-6

Get the __predictions__ from the test run.

In [29]:
test_run_details = test_run.get_details()
test_run_predictions = Dataset.get_by_id(ws, test_run_details['outputDatasets'][0]['identifier']['savedId'])
test_run_predictions.to_pandas_dataframe().head()

,Class_orig,Class_predicted,False_predicted_proba,True_predicted_proba,Time_orig,V1_orig,V2_orig,V3_orig,V4_orig,V5_orig,...,V20_orig,V21_orig,V22_orig,V23_orig,V24_orig,V25_orig,V26_orig,V27_orig,V28_orig,Amount_orig
0,False,False,0.75,0.25,144339.00,0.84,1.25,-2.01,0.46,0.53,...,0.51,-1.17,0.78,-0.07,-0.32,0.66,-0.05,0.04,0.16,1.00
1,False,False,0.91,0.09,99102.00,1.71,-0.37,-2.51,0.50,0.58,...,0.25,0.28,0.82,-0.41,0.13,0.68,0.39,-0.17,-0.05,237.00
2,False,False,0.89,0.11,128642.00,-4.10,-6.43,-0.24,1.13,-5.27,...,4.25,1.35,0.49,4.50,-0.14,0.60,0.64,-0.68,0.15,1895.62
3,False,False,0.94,0.06,31158.00,-1.54,0.22,1.19,-0.79,0.73,...,-0.35,-0.08,-0.10,-0.06,0.08,-0.87,0.65,-0.95,0.24,1.00
4,False,False,0.92,0.08,66866.00,-0.38,1.05,1.26,0.02,0.15,...,0.03,-0.25,-0.63,0.02,0.00,-0.26,0.10,0.26,0.09,0.89


## Test the fitted model remotely (On-demand)
This is testing a model the same way it was tested at the end of the training. However, in this case it's performed on-demand, meaning that you can test the model at any time, not just right after training.

In [30]:
from azureml.train.automl.model_proxy import ModelProxy

model_proxy = ModelProxy(best_run)
predictions, test_run_metrics = model_proxy.test(test_data)

print(predictions.to_pandas_dataframe().head())
pd.DataFrame.from_dict(test_run_metrics, orient='index', columns=['Value'])

   Class_orig  Class_predicted  False_predicted_proba  True_predicted_proba  \
0  False       False           0.93                   0.07                    
1  False       False           0.91                   0.09                    
2  False       False           0.91                   0.09                    
3  False       False           0.93                   0.07                    
4  False       False           0.86                   0.14                    

   Time_orig  V1_orig  V2_orig  V3_orig  V4_orig  V5_orig  ...  V20_orig  \
0 1.00       -0.97    -0.19    1.79     -0.86    -0.01     ... -0.21       
1 10.00      1.45     -1.18    0.91     -1.38    -1.97     ... -0.39       
2 11.00      1.07     0.29     0.83     2.71     -0.18     ... -0.15       
3 13.00      -0.44    0.92     0.92     -0.73    0.92      ... -0.05       
4 14.00      -5.40    -5.45    1.19     1.74     3.05      ... -2.20       

   V21_orig  V22_orig  V23_orig  V24_orig  V25_orig  V26_orig  V27_o

,Value
recall_score_weighted,1.00
f1_score_micro,1.00
accuracy,1.00
precision_score_macro,0.89
f1_score_weighted,1.00
precision_score_micro,1.00
average_precision_score_micro,1.00
balanced_accuracy,0.92
log_loss,0.09
recall_score_macro,0.92


## Test the fitted model locally

Now that the model is trained, split the data in the same way the data was split for training (The difference here is the data is being split locally) and then run the test data through the trained model to get the predicted values.

In [ ]:
# convert the test data to dataframe
X_test_df = test_data.drop_columns(columns=[label_column_name]).to_pandas_dataframe()
y_test_df = test_data.keep_columns(columns=[label_column_name], validate=True).to_pandas_dataframe()

In [ ]:
# call the predict functions on the model
y_pred = fitted_model.predict(X_test_df)
y_pred

### Calculate metrics for the prediction

Now visualize the data on a scatter plot to show what our truth (actual) values are compared to the predicted values 
from the trained model that was returned.

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

cf =confusion_matrix(y_test_df.values,y_pred)
plt.imshow(cf,cmap=plt.cm.Blues,interpolation='nearest')
plt.colorbar()
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
class_labels = ['False','True']
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks,class_labels)
plt.yticks([-0.5,0,1,1.5],['','False','True',''])
# plotting text value inside cells
thresh = cf.max() / 2.
for i,j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
    plt.text(j,i,format(cf[i,j],'d'),horizontalalignment='center',color='white' if cf[i,j] >thresh else 'black')
plt.show()

## Acknowledgements

This Credit Card fraud Detection dataset is made available under the Open Database License: http://opendatacommons.org/licenses/odbl/1.0/. Any rights in individual contents of the database are licensed under the Database Contents License: http://opendatacommons.org/licenses/dbcl/1.0/ and is available at: https://www.kaggle.com/mlg-ulb/creditcardfraud

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection.
More details on current and past projects on related topics are available on https://www.researchgate.net/project/Fraud-detection-5 and the page of the DefeatFraud project

Please cite the following works:

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019